In [1]:
import pandas as pd

In [7]:
working_dif = "/sfs/gpfs/tardis/project/bii_nssac/people/bl4zc/pgcoe_genepi_analytics"
ll_file = f"{working_dif}/data/initial_linelist.csv.xz"

df = pd.read_csv(ll_file,compression='xz')


In [8]:
df.sample(7)

,virus,region,country,division,divisionExposure,date,strain,sim_pid,contact_pid,sim_tick,...,vehicles,hh_income,household_language,family_type_and_employment_status,workers_in_family,rlid,asymptomatic,test_prob,tested_positive,exit_state
753847,ncov,North America,USA,Virginia,Virginia,2021-08-31,NaN,364776081,364607686,274,...,3,141000,1,2.0,1.0,1123433237,False,0.63,1,I2_a
357703,ncov,North America,USA,Virginia,Virginia,2021-08-06,NaN,362800970,362794635,249,...,2,89000,2,5.0,1.0,1125232861,False,1.00,1,hM2_s
617128,ncov,North America,USA,Virginia,Virginia,2021-08-21,NaN,366210435,366143414,264,...,2,137900,1,2.0,1.0,1122831674,False,0.63,1,I2_a
219166,ncov,North America,USA,Virginia,Virginia,2021-07-29,NaN,362749533,362574998,241,...,3,235700,1,1.0,3.0,1124083587,False,0.63,1,I2_a
1228195,ncov,North America,USA,Virginia,Virginia,2021-11-14,NaN,362547670,361433702,349,...,1,53200,1,NaN,NaN,1122100921,False,1.00,1,hM2_o
10272,ncov,North America,USA,Virginia,Virginia,2021-06-23,NaN,364108640,364108642,205,...,2,73800,5,1.0,2.0,1123446928,False,1.00,1,dM2_o
567217,ncov,North America,USA,Virginia,Virginia,2021-08-18,NaN,363635156,363635155,261,...,3,191000,1,1.0,2.0,1121755664,False,1.00,1,I2_s
